In [1]:
import pandas as pd

import techminer.data as data

filepath = "../data/papers/urban-agriculture.csv"
df = pd.read_csv(filepath)
df = data.load_scopus(df)

2020-06-04 21:21:31,874 - INFO - Renaming and selecting columns ...
2020-06-04 21:21:37,445 - INFO - Formatting author names ...
2020-06-04 21:21:37,453 - INFO - Disambiguating author names ...
2020-06-04 21:21:37,513 - INFO - Removing part of titles in foreing languages ...
2020-06-04 21:21:37,516 - INFO - Fusioning author and index keywords ...
2020-06-04 21:21:37,527 - INFO - NumExpr defaulting to 8 threads.
2020-06-04 21:21:37,542 - INFO - Extracting countries from affiliations ...
2020-06-04 21:21:43,807 - INFO - Extracting institutions from affiliations ...
2020-06-04 21:21:43,818 - INFO - Extracting country of 1st author ...
2020-06-04 21:21:43,821 - INFO - Extracting affiliation of 1st author ...
2020-06-04 21:21:43,824 - INFO - Counting number of authors ...


In [ ]:
data.coverage(df)

### Keywords

### FindString Dialog Box pag. 69

### Cleaning a list pag. 102

### Fuzzy match editor pag.126

### Stemming And / Stemming Or

### APPS

In [ ]:
import techminer.by_year as year
year.app(df)

In [ ]:
import techminer.by_term as term
term.app(df)

In [ ]:
import techminer.by_term_per_year as term_year
term_year.app(df)

In [ ]:
import techminer.co_occurrence as co_occurrence
co_occurrence.app(df)

In [ ]:
import techminer.correlation as correlation
correlation.app(df)

In [ ]:
import ipywidgets as widgets
from ipywidgets import Layout, AppLayout

In [ ]:

COLUMNS = [
    "Author Keywords",
    "Authors",
    "Countries",
    "Country 1st",
    "Document type",
    "Index Keywords",
    "Institution 1st",
    "Institutions",
    "Keywords",
    "Source title",
]

In [ ]:
FIGSIZE = (14, 10.0) ## Co-occurrente-analysis  <---- no tiene figura
FIGSIZE = (14, 9.0)

# NUEVA INTERFAZ

In [ ]:
from techminer.summary_by_term_per_year import documents_by_term_per_year, citations_by_term_per_year

In [ ]:
import techminer.plots as plt
from techminer.plots import COLORMAPS
from techminer.summary_by_term import citations_by_term, documents_by_term

WIDGET_WIDTH = "200px"
LEFT_PANEL_HEIGHT = "588px"
RIGHT_PANEL_WIDTH = "870px"
FIGSIZE = (14, 10.0)
PANE_HEIGHTS = ["80px", "650px", 0]

COLUMNS = [
    "Author Keywords",
    "Authors",
    "Countries",
    "Country 1st",
    "Document type",
    "Index Keywords",
    "Institution 1st",
    "Institutions",
    "Keywords",
    "Source title",
]



In [3]:
#################

import ipywidgets as widgets
from ipywidgets import AppLayout, Layout

from techminer.correlation import corr, corr_map
from techminer.plots import COLORMAPS
from techminer.plots import chord_diagram
from techminer.by_term import summary_by_term

WIDGET_WIDTH = "200px"
LEFT_PANEL_HEIGHT = "700px"
RIGHT_PANEL_WIDTH = "870px"
FIGSIZE = (14, 10.0)
PANE_HEIGHTS = ["80px", "750px", 0]

COLUMNS = [
    "Author Keywords",
    "Authors",
    "Countries",
    "Country 1st",
    "Document type",
    "Index Keywords",
    "Institution 1st",
    "Institutions",
    "Keywords",
    "Source title",
]


#################


#
#
#  Correlation Analysis
#
#
def body_0(x):
    #
    def server(**kwargs):
        #
        # Logic
        #
        column = kwargs["term"]
        by = kwargs["by"]
        method = kwargs["method"]
        min_link_value = kwargs["min_link_value"]
        cmap = kwargs["cmap"]
        filter_by = kwargs["filter_by"]
        filter_value = kwargs["filter_value"]
        view = kwargs["view"]
        #
        s = summary_by_term(x, column)
        if filter_by == "Frequency" and controls[-1]['widget'].max != s["Num Documents"].max():
            controls[-1]['widget'].max = s["Num Documents"].max()
            controls[-1]['widget'].value = 1
        if filter_by == "Cited by" and controls[-1]['widget'].max != s["Cited by"].max():
            controls[-1]['widget'].max = s["Cited by"].max()
            controls[-1]['widget'].value = 0
        #
        matrix = corr(
            x,
            column=column,
            by=by,
            method=method,
            min_link_value=min_link_value,
            cmap=cmap,
            filter_by=filter_by,
            filter_value=filter_value,
            as_matrix=True,
            keywords=None,
        )
        #
        if filter_by == "Frequency":
            s = s[[column, "Num Documents"]]
            d = {}
            for row in s.iterrows():
                d[row[1][column]] = row[1][column] +' [' + str(row[1]["Num Documents"]) + ']'
            matrix.columns = [d[w] for w in matrix.columns]
            matrix.index = [d[w] for w in matrix.index]
        if filter_by == "Cited by":
            s = s[[column, "Cited by"]]
            d = {}
            for row in s.iterrows():
                d[row[1][column]] = row[1][column] +' [' + str(row[1]["Cited by"]) + ']'
            matrix.columns = [d[w] for w in matrix.columns]
            matrix.index = [d[w] for w in matrix.index]

        #
        output.clear_output()
        with output:
            if view == "Matrix":
                if len(matrix.columns) <= 50 and len(matrix.index) <= 50:
                    display(matrix.style.background_gradient(cmap=cmap))
                else:
                    display(matrix)
            if view == "Correlatio map":
                pass
            if view == "Chord diagram":
                display(chord_diagram(matrix))

    #
    # UI
    #
    controls = [
        {
            "arg": "term",
            "desc": "Term to analyze:",
            "widget": widgets.Select(
                options=[z for z in COLUMNS if z in x.columns],
                ensure_option=True,
                disabled=False,
                layout=Layout(width=WIDGET_WIDTH),
            ),
        },
        {
            "arg": "by",
            "desc": "By Term:",
            "widget": widgets.Select(
                options=[z for z in COLUMNS if z in x.columns],
                ensure_option=True,
                disabled=False,
                layout=Layout(width=WIDGET_WIDTH),
            ),
        },
        {
            "arg": "method",
            "desc": "Method:",
            "widget": widgets.Dropdown(
                options=["pearson", "kendall", "spearman"],
                ensure_option=True,
                disabled=False,
                continuous_update=True,
                layout=Layout(width=WIDGET_WIDTH),
            ),
        },
        {
            "arg": "min_link_value",
            "desc": "Min link value:",
            "widget": widgets.FloatSlider(
                value=-1,
                min=-1,
                max=1.0,
                step=0.01,
                disabled=False,
                continuous_update=False,
                orientation="horizontal",
                readout=True,
                readout_format=".1f",
                layout=Layout(width=WIDGET_WIDTH),
            ),
        },     
        {
            "arg": "view",
            "desc": "View:",
            "widget": widgets.Dropdown(
                options=["Matrix", "Correlation map", "Chord diagram"],
                ensure_option=True,
                disabled=False,
                continuous_update=True,
                layout=Layout(width=WIDGET_WIDTH),
            ),
        },        
        {
            "arg": "cmap",
            "desc": "Matrix colormap:",
            "widget": widgets.Dropdown(
                options=COLORMAPS, disable=False, layout=Layout(width=WIDGET_WIDTH),
            ),
        },
        {
            "arg": "filter_by",
            "desc": "Filter by:",
            "widget": widgets.Dropdown(
                options=["Frequency", "Cited by"], disable=False, layout=Layout(width=WIDGET_WIDTH),
            ),
        },        
        {
            "arg": "filter_value",
            "desc": "Filter value",
            "widget": widgets.IntSlider(
                value=0,
                min=0,
                max=50,
                step=1,
                disabled=False,
                continuous_update=False,
                orientation="horizontal",
                readout=True,
                readout_format="d",
                layout=Layout(width=WIDGET_WIDTH),
            ),
        },
    ]
    #
    args = {control["arg"]: control["widget"] for control in controls}
    output = widgets.Output()
    with output:
        display(widgets.interactive_output(server, args,))
    return widgets.HBox(
        [
            widgets.VBox(
                [
                    widgets.VBox(
                        [widgets.Label(value=control["desc"]), control["widget"]]
                    )
                    for control in controls
                ],
                layout=Layout(height=LEFT_PANEL_HEIGHT, border="1px solid gray"),
            ),
            widgets.VBox([output], layout=Layout(width=RIGHT_PANEL_WIDTH)),
        ]
    )


#
#
# APP 
#
#

def app(df):
    #
    body = widgets.Tab()
    body.children = [body_0(df)]
    body.set_title(0, "Matrix")
    #
    return AppLayout(
        header=widgets.HTML(
            value="<h1>{}</h1><hr style='height:2px;border-width:0;color:gray;background-color:gray'>".format(
                "Correlation Analysis"
            )
        ),
        center=body,
        pane_heights=PANE_HEIGHTS,
    )


#####
app(df)

AppLayout(children=(HTML(value="<h1>Correlation Analysis</h1><hr style='height:2px;border-width:0;color:gray;b…

In [ ]:
from ipywidgets import interact, interactive, fixed, interact_manual
interact(slow_function,i=widgets.FloatSlider(min=1e5, max=1e7, step=1e5, continuous_update=True));

In [ ]:
def slow_function(i):
    print(int(i),list(x for x in range(int(i)) if
                str(x)==str(x)[::-1] and
                str(x**2)==str(x**2)[::-1]))
    return

In [ ]:
x = "hola"
x[:-1]

In [ ]:
import pandas as pd
x = pd.DataFrame({'a': [1, 2, 3, 4, 5], 'b': list('ABCDE')})
for row in x.iterrows():
    print(row[1]['b'])